## Goals

- Open a jwst file using the datamodel API
- Save the file as asdf
- Use tools for looking at the file and searching for attributes

The file jwst.asdf is presumed to be in the tutorial's data/ directory. This is the combined resampled product of two JWST NIRCam observations.

In [ ]:
import numpy as np
import asdf

from matplotlib import pyplot as plt
%matplotlib widget

JWST Data Models
----------------

JWST data files are typically stored in the FITS file format but internally have an embeded ASDF tree. The [jwst calibration pipeline](https://jwst-pipeline.readthedocs.io/en/latest/) uses a "data model" interface provided by the [stdatamodels](https://stdatamodels.readthedocs.io/en/latest/) package to read and write files. This interface can be used to convert JWST FITS files to pure ASDF which we'll use for this tutorial.

Let's start by using stdatamodels to open a JWST FITS file.

In [ ]:
import stdatamodels.jwst.datamodels as dm
model = dm.open("../data/jwst.fits")
model

Model data and metadata can be accessed as attributes (which may be nested).

In [ ]:
# to get the instrument name, access the meta attribute, then instrument sub-attribute, then finally the name
model.meta.instrument.name

For this tutorial we'll save this data model as a pure asdf file using `save`.

In [ ]:
model.save("jwst.asdf")

Now we can open the file with `asdf.open`.

In [ ]:
import asdf
af = asdf.open("jwst.asdf")

The file contents can be rendered using the **info** method

In [ ]:
af.info()

As the metadata can be richly nested it's often helpful to use `search` to look for keys matching the provided pattern.

In [ ]:
# Search for attributes with exposure in their name
af.search('exposure')

## Image data
This file contains a resampled image stored under the "data" key which we can display in this notebook.

In [ ]:
# Access the "data" under the "roman" sub-tree
im = af['data']
# Plot the data (with restricted values selected for this file)
fig, ax = plt.subplots()
ax.imshow(im, vmin=0, vmax=5, origin='lower')

## Array data
Arrays loaded from the file can be used in the same way as `numpy.ndarray`s. Let's take the log of the "data" array.

In [ ]:
# let's take a log of the image
log_image = af['data'].copy()
# filling all nan values
log_image[np.isnan(log_image)] = 5
# and clipping values <=0 to avoid issues with the log
log_image[log_image <= 0] = 1
log_image = np.log(log_image)
# and display the result
fig, ax = plt.subplots()
ax.imshow(im, vmin=0, vmax=2.5, origin='lower')

## Adding metadata
New values can be added to the ASDF tree using the dictionary interface.

In [ ]:
# Create new meta attribute
af['meta']['inspected_by'] = 'Jacques Clouseau'

Exercises
---------

1. Use `search` to find what "filter" was used to acquire this data.
2. Search for the name of the PI ("Klaus") and replace the attribute containing
   it with your own name. Type "af.search?" to show how to search values instead of attribute names.
3. Assign the `log_image` computed above to the "data" array.
5. Save the result to a new file named "jwst_exercise.asdf"
6. Load that new file and display the data image (verifying that the `log_image` was saved)
7. Print out the value of the attribute that contains the name of the PI
   to confirm the edits were saved